## Prepare Notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install datasets

In [ ]:
# --------------------- Imports ------------------------- #

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision.models import vit_b_16
import sys

# Datasets
# from datasets import load_dataset
import kagglehub

# Extra
import os
from copy import deepcopy
from PIL import Image
import matplotlib.pyplot as plt
import timm

## Office-31

In [ ]:
def Office31(domain, transform):
    path = kagglehub.dataset_download("xixuhu/office31")
    path = os.path.join(os.path.join(path, "Office-31"), domain)
    return datasets.ImageFolder(root=path, transform=transform)

def get_office_data_loaders(batch_size):

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    amazon_data = Office31("amazon", transform)
    dslr_data = Office31("dslr", transform)
    webcam_data = Office31("webcam", transform)

    loader_amazon = DataLoader(amazon_data, batch_size=batch_size, shuffle=True)
    loader_dslr = DataLoader(dslr_data, batch_size=batch_size, shuffle=True)
    loader_webcam = DataLoader(webcam_data, batch_size=batch_size, shuffle=True)

    return loader_amazon, loader_dslr, loader_webcam

## Pacs

## Model Cards

In [ ]:
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        base_model = torchvision.models.resnet50(pretrained=pretrained)
        self.conv1 = base_model.conv1
        self.bn1 = base_model.bn1
        self.relu = base_model.relu
        self.maxpool = base_model.maxpool
        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        f_block1 = self.layer1(x)
        f_block2 = self.layer2(f_block1)
        f_block3 = self.layer3(f_block2)
        f_block4 = self.layer4(f_block3)

        return f_block2, f_block3, f_block4

class VitBasicFeatureExtractor(nn.Module):

    def __init__(self, pretrained=True, layers=[4, 8, 12]):
        super().__init__()

        # self.model = timm.create_model('vit_base_patch16_224.sam_in1k', pretrained=pretrained, features_only=True)
        self.model = timm.create_model('vit_base_patch16_224', pretrained=pretrained, features_only=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):

        features = self.model(x)
        pooled_features = []
        for feature in features:
            pooled = self.avgpool(feature)
            pooled = pooled.view(pooled.size(0), -1)
            pooled_features.append(pooled)

        return tuple(pooled_features)


In [ ]:
model = VitBasicFeatureExtractor(pretrained=True)

input = torch.randn(16, 3, 224, 224)
output = model(input)

for out in output:
    print(out.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

torch.Size([16, 768])
torch.Size([16, 768])
torch.Size([16, 768])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init

class SparseAutoencoder(nn.Module):

    def __init__(self, input_dim=2048, hidden_dim=1024, latent_dim=768, dropout_rate=0.3):
        super(SparseAutoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_dim, latent_dim),
            nn.LayerNorm(latent_dim),
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # Kaiming initialization for layers followed by ReLU
                init.kaiming_uniform_(m.weight, a=0, nonlinearity='relu')
                if m.bias is not None:
                    init.zeros_(m.bias)
            elif isinstance(m, nn.LayerNorm):
                # Initialize LayerNorm with weight=1 and bias=0
                init.ones_(m.weight)
                init.zeros_(m.bias)

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z


In [ ]:
class BaselineModel(nn.Module):
    def __init__(self, feature_extractor, num_classes=7):

        super().__init__()
        self.feature_extractor = feature_extractor
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Linear(768, num_classes)

    def forward(self, x):
        _, _, f4 = self.feature_extractor(x)
        logits = self.classifier(f4)
        return logits, f4

In [ ]:
class AttentionAggregator(nn.Module):

    def __init__(self, embed_dim=768, num_heads=4, num_layers=1):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=embed_dim*4,
            dropout=0.1,
            activation='relu'
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, z_list):

        z_seq = torch.stack(z_list, dim=1)

        z_seq = z_seq.transpose(0, 1)

        attn_out = self.transformer_encoder(z_seq)

        attn_out = attn_out.transpose(0, 1)
        attn_out = self.norm(attn_out)

        # out = attn_out.reshape(attn_out.size(0), -1)  # (B, 3*embed_dim)
        out = torch.mean(attn_out, dim=1)

        return out

In [ ]:
class GatingAggregator(nn.Module):

    def __init__(self, embed_dim=768, num_input=3):
        super().__init__()
        self.alpha = nn.Parameter(torch.ones(num_input))

    def forward(self, z_list):
        alpha = torch.sigmoid(self.alpha)
        z_stack = torch.stack(z_list, dim=2)
        alpha_reshaped = alpha.view(1, 1, -1)
        z_gated = (z_stack * alpha_reshaped).sum(dim=2)
        return z_gated

In [ ]:
class UnifiedModelMultiBlockSAE(nn.Module):
    def __init__(self,
                 sae_dim_block2=128,
                 sae_dim_block3=256,
                 sae_dim_block4=256,
                 num_classes=31):
        super().__init__()

        self.feature_extractor = VitBasicFeatureExtractor(pretrained=True)

        self.sae2 = SparseAutoencoder(input_dim=768, latent_dim=768)
        self.sae3 = SparseAutoencoder(input_dim=768, latent_dim=768)
        self.sae4 = SparseAutoencoder(input_dim=768, latent_dim=768)

        # self.attention_aggregator = AttentionAggregator(embed_dim=768, num_heads=12, num_layers=3)
        self.gating_aggregator = GatingAggregator(embed_dim=768, num_input=2)

        self.classifier = nn.Sequential(
            nn.Linear(768, num_classes)
        )

    def forward(self, x):
        _, _, f4 = self.feature_extractor(x)

        # x_recon2, z2 = self.sae2(f2)
        # x_recon3, z3 = self.sae3(f3)
        x_recon4, z4 = self.sae4(f4)


        # attn_out = self.attention_aggregator([z2, z3, z4])
        # attn_out = self.gating_aggregator([z2, z3, z4])
        # attn_out = self.gating_aggregator([z4])

        class_logits = self.classifier(z4)

        return (class_logits,
                (x_recon4, z4),
                (f4))

In [ ]:
class UnifiedModelMultiBlockSAE_FFTAT(nn.Module):
    def __init__(self,
                 sae_dim_block2=128,
                 sae_dim_block3=256,
                 sae_dim_block4=256,
                 num_classes=31):
        super().__init__()

        self.feature_extractor = VitBasicFeatureExtractor(pretrained=True)

        self.sae2 = SparseAutoencoder(input_dim=768, latent_dim=768)
        self.sae3 = SparseAutoencoder(input_dim=768, latent_dim=768)
        self.sae4 = SparseAutoencoder(input_dim=768, latent_dim=768)

        # self.attention_aggregator = AttentionAggregator(embed_dim=768, num_heads=12, num_layers=3)
        self.gating_aggregator = GatingAggregator(embed_dim=768, num_input=2)

        self.classifier = nn.Sequential(
            nn.Linear(768, num_classes)
        )

    def forward(self, x_s, x_t=None, ad_net=None, cp_mask=None, optimal_flag=0):
        """
        We need to handle source and target images if x_t is not None.
        We also want to be consistent with how 'main.py' expects outputs:
        - logits_s
        - logits_t
        - local adversarial loss (maybe? or we just return 0)
        - reconstruction loss
        - final features for domain adv, etc.

        We'll do something like:
        """
        # extract features from source
        f2_s, f3_s, f4_s = self.feature_extractor(x_s)   # (B, 768) each, for example
        x_recon2_s, z2_s = self.sae2(f2_s)
        x_recon3_s, z3_s = self.sae3(f3_s)
        x_recon4_s, z4_s = self.sae4(f4_s)
        agg_s = self.gating_aggregator([z2_s, z3_s, z4_s])
        logits_s = self.classifier(agg_s)

        if x_t is not None:
            # do the same for target
            f2_t, f3_t, f4_t = self.feature_extractor(x_t)
            x_recon2_t, z2_t = self.sae2(f2_t)
            x_recon3_t, z3_t = self.sae3(f3_t)
            x_recon4_t, z4_t = self.sae4(f4_t)
            agg_t = self.gating_aggregator([z2_t, z3_t, z4_t])
            logits_t = self.classifier(agg_t)
        else:
            logits_t = None

        # if FFTAT wants local adv loss, you could do:
        #   e.g. loss_ad_local = something with ad_net?
        loss_ad_local = 0.0  # placeholder
        # we can compute an average reconstruction loss
        loss_rec = 0.0
        if x_t is not None:
            loss_rec_s = F.mse_loss(x_recon2_s, f2_s) + ...
            loss_rec_t = F.mse_loss(x_recon2_t, f2_t) + ...
            loss_rec = (loss_rec_s + loss_rec_t) * 0.5
        else:
            loss_rec = F.mse_loss(x_recon2_s, f2_s) + ...

        # Return the same structure that main.py expects
        #   (logits_s, logits_t, loss_ad_local, loss_rec, "source features", "target features", updated cp_mask?)
        return logits_s, logits_t, loss_ad_local, loss_rec, (f2_s, f3_s, f4_s), (f2_t, f3_t, f4_t), cp_mask

## Losses

In [ ]:
def irm_penalty(logits, labels):

    scale = torch.tensor(1.0, requires_grad=True, device=logits.device)
    loss_erm = F.cross_entropy(scale * logits, labels)

    grad = torch.autograd.grad(loss_erm, [scale], create_graph=True)[0]

    penalty = torch.sum(grad**2)
    var = 0.0

    return loss_erm, penalty, var

In [ ]:
def compute_sae_loss(recons, pools, zs, lambda_sparse=[1.0, 1.0, 1.0], lambda_reconstruction=[1.0, 1.0, 1.0]):

    x2_recon, x3_recon, x4_recon = recons
    f2p, f3p, f4p = pools
    z2, z3, z4 = zs

    rec_loss2 = F.mse_loss(x2_recon, f2p)
    rec_loss3 = F.mse_loss(x3_recon, f3p)
    rec_loss4 = F.mse_loss(x4_recon, f4p)

    lambda_r1, lambda_r2, lambda_r3 = lambda_reconstruction
    recon_loss_total = (lambda_r1 * rec_loss2 + lambda_r2 * rec_loss3 + lambda_r3 * rec_loss4)

    l1_z2 = torch.mean(torch.abs(z2))
    l1_z3 = torch.mean(torch.abs(z3))
    l1_z4 = torch.mean(torch.abs(z4))
    lambda_s1, lambda_s2, lambda_s3 = lambda_sparse
    l1_sparsity_total = (lambda_s1 * l1_z2 + lambda_s2 * l1_z3 + lambda_s3 * l1_z4)

    return recon_loss_total, l1_sparsity_total


In [ ]:
# @title MMD Loss

# Define Kernel
def gaussian_rbf_kernel(x1, x2, sigma):
    """
    Returns:
      A (B1 x B2) Tensor of pairwise RBF kernel values.
    """
    dist_sq = torch.cdist(x1, x2, p=2).pow(2)
    k = torch.exp(-dist_sq / (2.0 * sigma**2))
    return k


def mmd_rbf(f_s, f_t, sigma=1.0):
    """
    Returns:
      Scalar MMD^2 loss (Tensor).
    """
    # Compute kernels
    K_ss = gaussian_rbf_kernel(f_s, f_s, sigma)
    K_tt = gaussian_rbf_kernel(f_t, f_t, sigma)
    K_st = gaussian_rbf_kernel(f_s, f_t, sigma)

    # Averages
    mmd = K_ss.mean() + K_tt.mean() - 2*K_st.mean()
    return mmd

In [ ]:
# @title Coral Loss

def coral_loss(f_s, f_t):
    """
    Returns:
      Scalar CORAL loss (Tensor).
    """
    # Source covariance
    # = (1 / (Ns-1)) * (f_s^T * f_s)
    # But we do it in batch form in PyTorch.
    # subtract mean (batch form)
    f_s = f_s - f_s.mean(dim=0, keepdim=True)
    f_t = f_t - f_t.mean(dim=0, keepdim=True)

    Ns = f_s.size(0)
    Nt = f_t.size(0)

    cov_s = (f_s.t() @ f_s) / (Ns - 1)
    cov_t = (f_t.t() @ f_t) / (Nt - 1)

    # Frobenius norm of difference
    coral = (cov_s - cov_t).pow(2).sum()
    # Optionally scale by the factor 1/(4*d^2), but that’s a matter of definition
    # d = f_s.size(1)
    # coral = coral / (4.0 * (d**2))

    return coral


## Trian and Evaluate Functions

### Train

In [ ]:
def train_main_irm_multi_sae_office(model,
                              loader_source,
                              test_loader,
                              num_epochs=20,
                              lr=1e-4,
                              lr_sae=1e-4,
                              lambda_irm=1.0,
                              lambda_sae_rec=1.0,
                              lambda_sae_sparse=1e-4,
                              lambda_sparse=[1.0, 1.0, 1.0],
                              lambda_reconstruction=[1.0, 1.0, 1.0],
                              lambda_irm_pair=[1.0, 1.0, 1.0],
                              device='cuda',
                              verbose=False):

    model.to(device)
    model.train()

    def sae_forward_splits(f2p, f3p, f4p):
        x_recon2, z2 = model.sae2(f2p)
        x_recon3, z3 = model.sae3(f3p)
        x_recon4, z4 = model.sae4(f4p)
        return x_recon2, z2, x_recon3, z3, x_recon4, z4

    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        source_iter = iter(loader_source)
        steps_per_epoch = len(source_iter)

        for step in range(steps_per_epoch):

            try:
                x_s, y_s = next(source_iter)
            except StopIteration:
                source_iter = iter(loader_source)
                x_s, y_s = next(source_iter)

            x_s, y_s = x_s.to(device), y_s.to(device)

            class_logits_s, _, _, _, _ = model(x_s)

            loss_erm_s, penalty_s, var_s = irm_penalty(class_logits_s, y_s)

            irm_loss = 0.5 * (loss_erm_s)
            irm_pen  = 0.5 * (penalty_s)


            w1, w2, w3 = lambda_irm_pair
            loss_irm = w1 * (irm_loss) + w2 * (lambda_irm * irm_pen) + w3 * var_s


            with torch.no_grad():
                _, (x_recon2_s, z2_s), (x_recon3_s, z3_s), (x_recon4_s, z4_s), (f2p_s, f3p_s, f4p_s) = model(x_s)

            x_recon2_s, z2_s, x_recon3_s, z3_s, x_recon4_s, z4_s = sae_forward_splits(f2p_s, f3p_s, f4p_s)

            lambda_s1, lambda_s2, lambda_s3 = lambda_sparse
            lambda_r1, lambda_r2, lambda_r3 = lambda_reconstruction

            rec_loss2_s = F.mse_loss(x_recon2_s, f2p_s)
            rec_loss2   = lambda_r1 * (rec_loss2_s)

            rec_loss3_s = F.mse_loss(x_recon3_s, f3p_s)
            rec_loss3   = lambda_r2 * (rec_loss3_s)

            rec_loss4_s = F.mse_loss(x_recon4_s, f4p_s)
            rec_loss4   = lambda_r3 * (rec_loss4_s)

            rec_loss_total = (rec_loss2 + rec_loss3 + rec_loss4)

            l1_2_s = torch.mean(torch.abs(z2_s))
            l1_2   = lambda_s1 * (l1_2_s)

            l1_3_s = torch.mean(torch.abs(z3_s))
            l1_3   = lambda_s2 * (l1_3_s)

            l1_4_s = torch.mean(torch.abs(z4_s))
            l1_4   = lambda_s3 * (l1_4_s)

            l1_sparsity = l1_2 + l1_3 + l1_4

            sae_loss = lambda_sae_rec * rec_loss_total + lambda_sae_sparse * l1_sparsity

            # Feature Alignment Strategy
            features_s = f4_s  # or whichever feature block you want to align
            features_t = f4_t

            mmd_value   = mmd_rbf(features_s, features_t, sigma=1.0)
            coral_value = coral_loss(features_s, features_t)

            loss = loss_irm + sae_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (step+1) % 40 == 0 and verbose:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{steps_per_epoch}], "
                      f"IRM Loss: {loss_irm.item():.4f}, SAE Loss: {sae_loss.item():.4f}")

        test_acc = evaluate(model, test_loader, device=device)
        print(f"** End of Epoch {epoch+1}/{num_epochs} | Test Accuracy: {test_acc:.2f}% **")

    return model

In [ ]:
def train_model_irm_sae_with_warmup_office(
    batch_size=16,
    num_warmup_epochs=5,
    num_main_epochs=20,
    lr=1e-4,
    lr_sae=1e-4,
    lambda_irm=1.0,
    lambda_sae_rec=1.0,
    lambda_sae_sparse=1e-4,
    device='cuda',
    loader=['A', 'W'],
    model=None,
    verbose=False,
    lambda_sparse=[1.0, 1.0, 1.0],
    lambda_reconstruction=[1.0, 1.0, 1.0],
    lambda_irm_pair=[1.0, 1.0, 0.0],
):

    loader_amazon, loader_webcam, loader_dslr = get_office_data_loaders(batch_size)

    source, target = loader if loader is not None else ['A', 'W']

    if source == 'A':
        loader_source = loader_amazon
    elif source == 'W':
        loader_source = loader_webcam
    elif source == 'D':
        loader_source = loader_dslr

    if target == 'A':
        loader_target = loader_amazon
    elif target == 'W':
        loader_target = loader_webcam
    elif target == 'D':
        loader_target = loader_dslr


    if model is None:
        model = UnifiedModelMultiBlockSAE(512, 1024, 2048, 31)

    print("===== Main Phase (IRM + SAE) =====")

    model = train_main_irm_multi_sae_office(model,
                                            loader_source=loader_source,
                                            test_loader=loader_target,
                                            num_epochs=num_main_epochs,
                                            lr=lr,
                                            lr_sae=lr_sae,
                                            lambda_irm=lambda_irm,
                                            lambda_sae_rec=lambda_sae_rec,
                                            lambda_sae_sparse=lambda_sae_sparse,
                                            device=device,
                                            verbose=verbose,
                                            lambda_sparse=lambda_sparse,
                                            lambda_reconstruction=lambda_reconstruction,
                                            lambda_irm_pair=lambda_irm_pair
                                            )

    return model

### Separate optm

In [ ]:
def train_main_irm_multi_sae_office(
    model,
    loader_source,
    test_loader,
    num_epochs=20,
    lr=1e-4,
    lr_sae=1e-4,
    lambda_irm=1.0,
    lambda_sae_rec=1.0,
    lambda_sae_sparse=1e-4,
    lambda_sparse=[1.0, 1.0, 1.0],
    lambda_reconstruction=[1.0, 1.0, 1.0],
    lambda_irm_pair=[1.0, 1.0, 1.0],
    device='cuda',
    verbose=False
):
    import torch.optim as optim

    model.to(device)
    model.train()

    def sae_forward_splits(f4p):
        # x_recon2, z2 = model.sae2(f2p)
        # x_recon3, z3 = model.sae3(f3p)
        x_recon4, z4 = model.sae4(f4p)
        return x_recon4, z4

    # Define separate optimizers
    # Optimizer for the feature extractor and classifier
    params_rest = [
        p for n, p in model.named_parameters()
        if not (n.startswith('sae2') or n.startswith('sae3') or n.startswith('sae4'))
    ]
    # optimizer_rest = optim.SGD(params_rest, lr=lr, momentum=0.9)
    optimizer_rest = optim.Adam(params_rest, lr=lr)
    # Optimizer for the Sparse Autoencoders (sae2, sae3, sae4)
    params_sae = list(model.sae2.parameters()) + list(model.sae3.parameters()) + list(model.sae4.parameters())
    optimizer_sae = optim.Adam(params_sae, lr=lr_sae)

    for epoch in range(num_epochs):
        source_iter = iter(loader_source)
        steps_per_epoch = len(source_iter)

        for step in range(steps_per_epoch):

            try:
                x_s, y_s = next(source_iter)
            except StopIteration:
                source_iter = iter(loader_source)
                x_s, y_s = next(source_iter)

            x_s, y_s = x_s.to(device), y_s.to(device)

            # Forward pass for classification
            class_logits_s, _, _ = model(x_s)

            # Compute IRM loss
            loss_erm_s, penalty_s, var_s = irm_penalty(class_logits_s, y_s)

            irm_loss = 0.5 * loss_erm_s
            irm_pen  = 0.5 * penalty_s

            w1, w2, w3 = lambda_irm_pair
            loss_irm = w1 * irm_loss + w2 * (lambda_irm * irm_pen) + w3 * var_s

            # Forward pass for SAEs without tracking gradients
            with torch.no_grad():
                class_logits_s, (x_recon4_s, z4_s), (f4p_s) = model(x_s)

            # Forward pass through SAEs to get reconstructions and latent vectors
            x_recon4_s, z4_s = sae_forward_splits(f4p_s)

            # Compute Reconstruction Loss
            lambda_s1, lambda_s2, lambda_s3 = lambda_sparse
            lambda_r1, lambda_r2, lambda_r3 = lambda_reconstruction


            rec_loss4_s = F.mse_loss(x_recon4_s, f4p_s)
            rec_loss4   = lambda_r3 * rec_loss4_s

            rec_loss_total = rec_loss4

            l1_4_s = torch.mean(torch.abs(z4_s))
            l1_4   = lambda_s3 * l1_4_s

            l1_sparsity = l1_4

            # Total SAE Loss
            sae_loss = lambda_sae_rec * rec_loss_total + lambda_sae_sparse * l1_sparsity

            loss = loss_irm + sae_loss


            # Zero gradients for both optimizers
            optimizer_rest.zero_grad()
            optimizer_sae.zero_grad()

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer_rest.step()
            optimizer_sae.step()

            if (step+1) % 40 == 0 and verbose:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{steps_per_epoch}], "
                      f"IRM Loss: {loss_irm.item():.4f}, SAE Loss: {sae_loss.item():.4f}")

        # Evaluation after each epoch
        test_acc = evaluate(model, test_loader, device=device)
        print(f"** End of Epoch {epoch+1}/{num_epochs} | Test Accuracy: {test_acc:.2f}% **")

    return model


In [ ]:
# @title Domain and Target Distribution Alignment
def train_office_with_mmd_coral(
    model,
    loader_source,
    loader_target,
    num_epochs=20,
    lr=1e-4,
    lambda_irm=1.0,
    lambda_sae=1.0,
    lambda_mmd=0.1,
    lambda_coral=0.0,
    device='cuda'
):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        source_iter = iter(loader_source)
        target_iter = iter(loader_target)
        steps_per_epoch = min(len(source_iter), len(target_iter))

        for step in range(steps_per_epoch):
            x_s, y_s = next(source_iter)
            x_t, _   = next(target_iter)

            x_s, y_s = x_s.to(device), y_s.to(device)
            x_t      = x_t.to(device)

            # Forward pass on source
            logits_s, (x_recon_s, z_s), (f4_s) = model(x_s)
            # If you want target features for alignment
            # no ground-truth labels for target
            _, (x_recon_t, z_t), (f4_t) = model(x_t)

            # IRM or ERM on source
            loss_erm_s, penalty_s, var_s = irm_penalty(logits_s, y_s)
            irm_loss = 0.5 * loss_erm_s
            irm_pen  = 0.5 * penalty_s
            loss_irm = irm_loss + lambda_irm * irm_pen

            # SAE loss
            sae_loss_source = F.mse_loss(x_recon_s, f4_s)  # + optional L1-sparsity
            loss_sae = lambda_sae * sae_loss_source

            # MMD or CORAL
            # only do it on f4_s, f4_t or aggregated features
            if lambda_mmd > 0:
                loss_mmd = mmd_rbf(f4_s, f4_t, sigma=1.0)
            else:
                loss_mmd = 0.0

            if lambda_coral > 0:
                loss_coral = coral_loss(f4_s, f4_t)
            else:
                loss_coral = 0.0

            total_loss = loss_irm + loss_sae + lambda_mmd * loss_mmd + lambda_coral * loss_coral

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

        # Evaluate on target after epoch
        acc = evaluate(model, loader_target, device=device)
        print(f"Epoch {epoch+1}/{num_epochs} - Target Acc: {acc:.2f}%")

    return model

### Evaluate

In [ ]:
def evaluate_baseline(model, loader, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits, _ = model(x)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = 100.0 * correct / total
    model.train()
    return acc


In [ ]:
def evaluate(model, loader, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits, _, _ = model(x)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = 100.0 * correct / total
    return acc

# MAIN

In [ ]:
feature_extractor_baseline = ResNetFeatureExtractor(pretrained=True)
baseline_model = BaselineModel(feature_extractor=feature_extractor_baseline, num_classes=31)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 51.1MB/s]


In [ ]:
loader_amazon, loader_webcam, loader_dslr = get_office_data_loaders(32)

100%|██████████| 75.9M/75.9M [00:04<00:00, 17.7MB/s]

Extracting files...


## Replacing backbone of FFTAT with ours

In [ ]:
# @title Prepare FFTAT

# Git clone Repository
# !git clone https://github.com/Shawey94/WACV2025-FFTAT.git /content/drive/MyDrive/ATML/Project/FFTAT
%cd /content/drive/MyDrive/ATML/Project/FFTAT
sys.path.append('/content/drive/MyDrive/ATML/Project/FFTAT')

!pip install ml_collections cpnet

# Import necessary functions from FFTAT
from models.modeling import VisionTransformer, AdversarialNetwork  # or whatever classes you need
from models.lossZoo import im, adv, adv_local
from models.configs import get_b16_config

/content/drive/.shortcut-targets-by-id/1X7wTsg8g3Bx368F088lgtoQNou7sUSvm/ATML/Project/FFTAT
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
os.makedirs('./output/office', exist_ok=True)

In [ ]:
from main import setup, train

In [ ]:
# Prepare your dataloaders
loader_amazon, loader_dslr, loader_webcam = get_office_data_loaders(batch_size=16)

# Modify arguments
from argparse import Namespace

args = Namespace(
    train_batch_size=16,
    eval_batch_size=16,
    dataset='office',
    name='custom_run',
    num_classes=31,
    model_type='ViT-B_16',
    pretrained_dir='checkpoint/imagenet21k_ViT-B_16.npz',
    num_steps=1000,
    img_size=224,
    beta=1.0,
    gamma=0.1,
    use_im=True,
    theta=0.1,
    learning_rate=0.05,
    gpu_id='0',
    use_cp=True,
    optimal=1,
    warmup_steps=100,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    local_rank=-1,
    output_dir='./output/office',
    seed=42,
    gradient_accumulation_steps = 1,
    weight_decay=0.05,
    decay_type = 'cosine',
    n_gpu=torch.cuda.device_count(),
    index_t=None,
    max_grad_norm =1.0,
    eval_every=20
)

# Setup the model
args, model = setup(args, prefix_saved_mode='custom_prefix')
model.to(args.device)

# Call the train function directly with your dataloaders
train(
    args=args,
    model=model,
    cp_mask=torch.ones((257, 257)).float().to(args.device),  # Placeholder cp_mask
    prefix_saved_mode='custom_prefix',
    source_loader=loader_amazon,
    target_loader=loader_dslr,
    test_loader=loader_webcam
)

loss_clc 3.9182162284851074, loss_im -0.2223825454711914, loss_ad_global 1.1144098043441772, loss_ad_local 0.7492037415504456
loss_clc 4.0403151512146, loss_im -0.2519564628601074, loss_ad_global 1.3685094118118286, loss_ad_local 0.819049060344696
loss_clc 3.9631385803222656, loss_im -0.1955413818359375, loss_ad_global 1.2823833227157593, loss_ad_local 0.8071699142456055
loss_clc 3.3580057621002197, loss_im -0.2554030418395996, loss_ad_global 1.509061336517334, loss_ad_local 0.7298853397369385
loss_clc 3.448415756225586, loss_im -0.23902225494384766, loss_ad_global 1.4221951961517334, loss_ad_local 0.8075733184814453
loss_clc 3.9552886486053467, loss_im -0.21948623657226562, loss_ad_global 1.3271571397781372, loss_ad_local 0.7943347692489624
loss_clc 4.107184410095215, loss_im -0.20530223846435547, loss_ad_global 1.604514718055725, loss_ad_local 0.7931171655654907
loss_clc 3.7605388164520264, loss_im -0.27085065841674805, loss_ad_global 2.1063642501831055, loss_ad_local 0.8037132620811

Validating... (loss=3.91063): 100%|| 50/50 [00:07<00:00,  6.72it/s]


loss_clc 3.424018144607544, loss_im -0.1997818946838379, loss_ad_global 0.8191932439804077, loss_ad_local 0.7734259366989136
loss_clc 3.517679214477539, loss_im -0.19598627090454102, loss_ad_global 1.5956552028656006, loss_ad_local 0.8097792863845825
loss_clc 3.5613956451416016, loss_im -0.18513107299804688, loss_ad_global 1.3879836797714233, loss_ad_local 0.7632859349250793
loss_clc 3.3516435623168945, loss_im -0.2064228057861328, loss_ad_global 1.345506191253662, loss_ad_local 0.7414949536323547
loss_clc 3.4231584072113037, loss_im -0.16926860809326172, loss_ad_global 1.2168604135513306, loss_ad_local 0.772557258605957
loss_clc 3.556591510772705, loss_im -0.19895410537719727, loss_ad_global 1.0827441215515137, loss_ad_local 0.7288106679916382
loss_clc 3.8547465801239014, loss_im -0.2252969741821289, loss_ad_global 1.314551591873169, loss_ad_local 0.751552402973175
loss_clc 3.6790733337402344, loss_im -0.18972110748291016, loss_ad_global 1.1972688436508179, loss_ad_local 0.75109028816

Validating... (loss=2.92786): 100%|| 50/50 [00:07<00:00,  6.71it/s]


loss_clc 3.6577796936035156, loss_im -0.11170196533203125, loss_ad_global 0.9972925782203674, loss_ad_local 0.7585691213607788
loss_clc 3.515991449356079, loss_im -0.1024618148803711, loss_ad_global 1.1765656471252441, loss_ad_local 0.7533301115036011
loss_clc 3.328976631164551, loss_im -0.11107587814331055, loss_ad_global 0.7252852916717529, loss_ad_local 0.7492287755012512
loss_clc 3.5804476737976074, loss_im -0.09651708602905273, loss_ad_global 0.9410068988800049, loss_ad_local 0.7276219129562378
loss_clc 3.6424708366394043, loss_im -0.12490510940551758, loss_ad_global 1.0697402954101562, loss_ad_local 0.7515203356742859
loss_clc 3.01625919342041, loss_im -0.11713266372680664, loss_ad_global 0.9953427314758301, loss_ad_local 0.7305676937103271
loss_clc 3.4956040382385254, loss_im -0.0876460075378418, loss_ad_global 0.6784145832061768, loss_ad_local 0.7697339653968811
loss_clc 3.3623814582824707, loss_im -0.105804443359375, loss_ad_global 0.8750609159469604, loss_ad_local 0.759798109

...

loss_clc 3.4291980266571045, loss_im 0.0, loss_ad_global 0.6931185722351074, loss_ad_local 0.7198622226715088
loss_clc 3.4388253688812256, loss_im 0.0, loss_ad_global 0.6932035684585571, loss_ad_local 0.7179951667785645
loss_clc 3.457305431365967, loss_im 0.0, loss_ad_global 0.6931241750717163, loss_ad_local 0.7188999056816101
loss_clc 3.432013511657715, loss_im 0.0, loss_ad_global 0.6931202411651611, loss_ad_local 0.7202663421630859
loss_clc 3.4223241806030273, loss_im 0.0, loss_ad_global 0.6931560039520264, loss_ad_local 0.7203948497772217
loss_clc 3.434385299682617, loss_im 0.0, loss_ad_global 0.6931293606758118, loss_ad_local 0.722762942314148
loss_clc 3.4063758850097656, loss_im 0.0, loss_ad_global 0.6931552886962891, loss_ad_local 0.7220745086669922
loss_clc 3.432142972946167, loss_im 0.0, loss_ad_global 0.6932742595672607, loss_ad_local 0.7201595902442932
loss_clc 3.4251317977905273, loss_im 0.0, loss_ad_global 0.6931484341621399, loss_ad_local 0.7213200330734253
loss_clc 3.4177

Validating... (loss=3.41288): 100%|| 50/50 [00:07<00:00,  6.80it/s]


loss_clc 3.424133062362671, loss_im 0.0, loss_ad_global 0.6931602954864502, loss_ad_local 0.7198304533958435
loss_clc 3.406224012374878, loss_im 0.0, loss_ad_global 0.6931211948394775, loss_ad_local 0.7200573682785034
loss_clc 3.430391311645508, loss_im 0.0, loss_ad_global 0.6931997537612915, loss_ad_local 0.7204936742782593
loss_clc 3.429884672164917, loss_im 0.0, loss_ad_global 0.6931040287017822, loss_ad_local 0.7193065285682678
loss_clc 3.4171667098999023, loss_im 0.0, loss_ad_global 0.6931702494621277, loss_ad_local 0.7206051349639893
loss_clc 3.4048728942871094, loss_im 0.0, loss_ad_global 0.693095326423645, loss_ad_local 0.7189919948577881
loss_clc 3.4217944145202637, loss_im 0.0, loss_ad_global 0.6933175325393677, loss_ad_local 0.7223632335662842
loss_clc 3.4045252799987793, loss_im 0.0, loss_ad_global 0.6931214332580566, loss_ad_local 0.7214557528495789
loss_clc 3.4283647537231445, loss_im 0.0, loss_ad_global 0.6932305097579956, loss_ad_local 0.7202202081680298
loss_clc 3.4190

Validating... (loss=3.40695): 100%|| 50/50 [00:07<00:00,  6.82it/s]


loss_clc 3.447458028793335, loss_im 0.0, loss_ad_global 0.6932148933410645, loss_ad_local 0.7205572724342346
loss_clc 3.413261890411377, loss_im 0.0, loss_ad_global 0.693123459815979, loss_ad_local 0.7202247381210327
loss_clc 3.458590269088745, loss_im 0.0, loss_ad_global 0.6930873394012451, loss_ad_local 0.7215464115142822
loss_clc 3.450324058532715, loss_im 0.0, loss_ad_global 0.6931618452072144, loss_ad_local 0.7192971706390381
loss_clc 3.435222625732422, loss_im 0.0, loss_ad_global 0.6931302547454834, loss_ad_local 0.7191076278686523
loss_clc 3.409395933151245, loss_im 0.0, loss_ad_global 0.693221926689148, loss_ad_local 0.7194498181343079
loss_clc 3.426117181777954, loss_im 0.0, loss_ad_global 0.6931492686271667, loss_ad_local 0.7200491428375244
loss_clc 3.443356513977051, loss_im 0.0, loss_ad_global 0.6931293606758118, loss_ad_local 0.7185658812522888
loss_clc 3.4352004528045654, loss_im 0.0, loss_ad_global 0.693161129951477, loss_ad_local 0.7193993330001831
loss_clc 3.4441750049

In [ ]:
from google.colab import files

# uploaded = files.upload()

import tensorflow as tf

# Replace 'path_to_file' with your uploaded file's name
event_file = '/content/events.out.tfevents.1735305008.47c8fa70d046.5662.0'


In [ ]:

# Output text file
output_txt_file = 'events_parsed.txt'

with open(output_txt_file, 'w') as txt_file:
    for event in tf.compat.v1.train.summary_iterator(event_file):
        txt_file.write(f"Event timestamp: {event.wall_time}\n")
        for value in event.summary.value:
            txt_file.write(f"Tag: {value.tag}\n")
            if value.HasField('simple_value'):
                txt_file.write(f"Value: {value.simple_value}\n")
        txt_file.write("\n")

# Download the parsed text file
from google.colab import files
files.download(output_txt_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>